# Import the libraries

In [105]:
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

In [106]:
wb = load_workbook("PLC9_PLC3_BDC1_BDC2_12_wells.xlsx")

In [107]:
# -- Open worksheets for BDC1A --
# Well 1
ws_bdc1a_well1 = wb["BDC1A_WELL1"]
ws_bdc1a_well1_setpoint = wb["BDC1A_WELL1_SETPOINT"]

# Well 2
ws_bdc1a_well2 = wb["BDC1A_WELL2"]
ws_bdc1a_well2_setpoint = wb["BDC1A_WELL2_SETPOINT"]

# Well 3
ws_bdc1a_well3 = wb["BDC1A_WELL3"]
ws_bdc1a_well3_setpoint = wb["BDC1A_WELL3_SETPOINT"]

# Well 4
ws_bdc1a_well4 = wb["BDC1A_WELL4"]
ws_bdc1a_well4_setpoint = wb["BDC1A_WELL4_SETPOINT"]

# Well 5
ws_bdc1a_well5 = wb["BDC1A_WELL5"]
ws_bdc1a_well5_setpoint = wb["BDC1A_WELL5_SETPOINT"]

# Well 6
ws_bdc1a_well6 = wb["BDC1A_WELL6"]
ws_bdc1a_well6_setpoint = wb["BDC1A_WELL6_SETPOINT"]

# -- Open worksheets for BDC2 --
# Well 1
ws_bdc2_well1 = wb["BDC2_WELL1"]
ws_bdc2_well1_setpoint = wb["BDC2_WELL1_SETPOINT"]

# Well 2
ws_bdc2_well2 = wb["BDC2_WELL2"]
ws_bdc2_well2_setpoint = wb["BDC2_WELL2_SETPOINT"]

# Well 3
ws_bdc2_well3 = wb["BDC2_WELL3"]
ws_bdc2_well3_setpoint = wb["BDC2_WELL3_SETPOINT"]

# Well 4
ws_bdc2_well4 = wb["BDC2_WELL4"]
ws_bdc2_well4_setpoint = wb["BDC2_WELL4_SETPOINT"]

# Well 5
ws_bdc2_well5 = wb["BDC2_WELL5"]
ws_bdc2_well5_setpoint = wb["BDC2_WELL5_SETPOINT"]

# Well 6
ws_bdc2_well6 = wb["BDC2_WELL6"]
ws_bdc2_well6_setpoint = wb["BDC2_WELL6_SETPOINT"]

# BDC1A_WELL1

In [108]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-19" # Change this to respective well
regEx_UXCF = "UXCF"
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc1a_well1.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)


In [109]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "46|46_|46A_|46B_|52|52_|52A_|52B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "41|51" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [110]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-"
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [111]:
# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6
write_description_col = 16

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC1A_WELL1 Modbus sheet
for row in ws_bdc1a_well1.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc1a_well1_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc1a_well1_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx].value == "16 UNSIGNED" else "Float")
    ws_bdc1a_well1_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc1a_well1_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc1a_well1_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc1a_well1_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc1a_well1_setpoint.cell(row=write_row, column=write_description_col, value=service_name)


    # Increase the row by 1
    write_row += 1

# BDC1A_WELL2

In [112]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-29" # Change this to respective well
regEx_UXCF = "UXCF"
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc1a_well2.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)


In [113]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "246|246_|246A_|246B_|252|252_|252A_|252B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "241|251" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [114]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-|-1" # Change this to respective well
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [115]:
# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC1A_well2 Modbus sheet
for row in ws_bdc1a_well2.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc1a_well2_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc1a_well2_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx ].value == "16 UNSIGNED" else "Float")
    ws_bdc1a_well2_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc1a_well2_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc1a_well2_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc1a_well2_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc1a_well2_setpoint.cell(row=write_row, column=write_description_col, value=service_name)


    # Increase the row by 1
    write_row += 1

# BDC1A_WELL3

In [116]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-39" # Change this to respective well
regEx_UXCF = "UXCF"
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc1a_well3.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)


In [117]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "346|346_|346A_|346B_|352|352_|352A_|352B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "341|351" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [118]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-|-1" # Change this to respective well
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [119]:
# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC1A_well3 Modbus sheet
for row in ws_bdc1a_well3.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc1a_well3_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc1a_well3_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx ].value == "16 UNSIGNED" else "Float")
    ws_bdc1a_well3_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc1a_well3_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc1a_well3_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc1a_well3_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc1a_well3_setpoint.cell(row=write_row, column=write_description_col, value=service_name)


    # Increase the row by 1
    write_row += 1

# BDC1A_WELL4

In [120]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-49" # Change this to respective well
regEx_UXCF = "UXCF"
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc1a_well4.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)

In [121]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "446|446_|446A_|446B_|452|452_|452A_|452B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "441|451" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [122]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-|-1" # Change this to respective well
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [123]:
# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC1A_well4 Modbus sheet
for row in ws_bdc1a_well4.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc1a_well4_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc1a_well4_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx ].value == "16 UNSIGNED" else "Float")
    ws_bdc1a_well4_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc1a_well4_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc1a_well4_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc1a_well4_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc1a_well4_setpoint.cell(row=write_row, column=write_description_col, value=service_name)


    # Increase the row by 1
    write_row += 1

# BDC1A_WELL5

In [124]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-59" # Change this to respective well
regEx_UXCF = "UXCF" 
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc1a_well5.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)

In [125]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "546|546_|546A_|546B_|552|552_|552A_|552B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "541|551" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [126]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-|-1" # Change this to respective well
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [127]:
# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC1A_well5 Modbus sheet
for row in ws_bdc1a_well5.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc1a_well5_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc1a_well5_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx ].value == "16 UNSIGNED" else "Float")
    ws_bdc1a_well5_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc1a_well5_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc1a_well5_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc1a_well5_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc1a_well5_setpoint.cell(row=write_row, column=write_description_col, value=service_name)

    # Increase the row by 1
    write_row += 1

# BDC1A_WELL6

In [128]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-69" # Change this to respective well
regEx_UXCF = "UXCF" 
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc1a_well6.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)

In [129]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "646|646_|646A_|646B_|652|652_|652A_|652B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "641|651" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [130]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-|-1" # Change this to respective well
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [131]:
# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC1A_well6 Modbus sheet
for row in ws_bdc1a_well6.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc1a_well6_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc1a_well6_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx ].value == "16 UNSIGNED" else "Float")
    ws_bdc1a_well6_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc1a_well6_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc1a_well6_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc1a_well6_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc1a_well6_setpoint.cell(row=write_row, column=write_description_col, value=service_name)

    # Increase the row by 1
    write_row += 1

# BDC2_WELL1

In [132]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-19" # Change this to respective well
regEx_UXCF = "UXCF"
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc2_well1.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)

In [133]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "46|46_|46A_|46B_|52|52_|52A_|52B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "41|51" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [134]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-"
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [135]:
# Define columns index in modbus sheet
starting_row = 2
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC2_WELL1 Modbus sheet
for row in ws_bdc2_well1.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc2_well1_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc2_well1_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx ].value == "16 UNSIGNED" else "Float")
    ws_bdc2_well1_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc2_well1_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc2_well1_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc2_well1_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc2_well1_setpoint.cell(row=write_row, column=write_description_col, value=service_name)

    # Increase the row by 1
    write_row += 1

# BDC2_WELL2

In [136]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-29" # Change this to respective well
regEx_UXCF = "UXCF"
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc2_well2.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)


In [137]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "246|246_|246A_|246B_|252|252_|252A_|252B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "241|251" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [138]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-|-1" # Change this to respective well
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [139]:
# Define columns index in modbus sheet
starting_row = 2
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC2_well2 Modbus sheet
for row in ws_bdc2_well2.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc2_well2_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc2_well2_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx ].value == "16 UNSIGNED" else "Float")
    ws_bdc2_well2_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc2_well2_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc2_well2_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc2_well2_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc2_well2_setpoint.cell(row=write_row, column=write_description_col, value=service_name)

    # Increase the row by 1
    write_row += 1

# BDC2_WELL3


In [140]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-39" # Change this to respective well
regEx_UXCF = "UXCF"
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc2_well3.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)


In [141]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "346|346_|346A_|346B_|352|352_|352A_|352B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "341|351" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [142]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-|-1" # Change this to respective well
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [143]:
# Define columns index in modbus sheet
starting_row = 2
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC2_well3 Modbus sheet
for row in ws_bdc2_well3.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc2_well3_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc2_well3_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx ].value == "16 UNSIGNED" else "Float")
    ws_bdc2_well3_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc2_well3_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc2_well3_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc2_well3_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc2_well3_setpoint.cell(row=write_row, column=write_description_col, value=service_name)

    # Increase the row by 1
    write_row += 1

# BDC2_WELL4

In [144]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-49" # Change this to respective well
regEx_UXCF = "UXCF"
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc2_well4.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)

In [145]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "446|446_|446A_|446B_|452|452_|452A_|452B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "441|451" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [146]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-|-1" # Change this to respective well
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [147]:
# Define columns index in modbus sheet
starting_row = 2
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC2_well4 Modbus sheet
for row in ws_bdc2_well4.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc2_well4_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc2_well4_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx ].value == "16 UNSIGNED" else "Float")
    ws_bdc2_well4_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc2_well4_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc2_well4_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc2_well4_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc2_well4_setpoint.cell(row=write_row, column=write_description_col, value=service_name)

    # Increase the row by 1
    write_row += 1

# BDC2_WELL5

In [148]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-59" # Change this to respective well
regEx_UXCF = "UXCF" 
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc2_well5.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)

In [149]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "546|546_|546A_|546B_|552|552_|552A_|552B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "541|551" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [150]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-|-1" # Change this to respective well
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [151]:
# Define columns index in modbus sheet
starting_row = 2
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC2_well5 Modbus sheet
for row in ws_bdc2_well5.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc2_well5_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc2_well5_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx ].value == "16 UNSIGNED" else "Float")
    ws_bdc2_well5_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc2_well5_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc2_well5_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc2_well5_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc2_well5_setpoint.cell(row=write_row, column=write_description_col, value=service_name)

    # Increase the row by 1
    write_row += 1

# BDC2_WELL6

In [152]:
# Create a valve name set
# Define an expression for valve
regEx_valve = "-.{2,4}V-" 
regEx_valve_service = "VALVE"
regEx_zi = "ZI"
regEx_xy = "XY-69" # Change this to respective well
regEx_UXCF = "UXCF" 
valves_name_set = set() 

# Define columns index in modbus sheet
starting_row = 4
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 4
io_type_idx = 6
bit_number_address_idx = 7
for row in ws_bdc2_well6.iter_rows(min_row=starting_row):
    ss_tag = row[ss_tag_idx].value
    service = row[service_idx].value
    if re.search(regEx_UXCF, ss_tag):
        continue
    elif re.search(regEx_xy, ss_tag):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)
    elif re.search(regEx_valve, ss_tag) and re.search(regEx_valve_service, service):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        valves_name_set.add(device_name)

In [153]:
# Return the device name
regEx_asd = "-AI-|-VI-"
regEx_dhpt = "646|646_|646A_|646B_|652|652_|652A_|652B_" # Change this to respective well
regEx_pt = "-PI-|-TI-"
regEx_zi_gc_xc = "-ZI|-GC|-XC-"
regEx_dpi = "-DPI-"
regEx_04 = "04"
regEx_41_51 = "641|651" # Change this to respective well
regEx_ub = "-UB-"
regEx_un = "-UN-"
regEx_uxcfa = "-UXCF.*A_"
regEx_uxcfb = "-UXCF.*B_"
regEx_fzu = "-FZU-"
def create_device_name(ss_tag):
    device_name = ""

    # If the tag contains AI or VI, the device's name is ASD
    if re.search(regEx_asd, ss_tag):
        device_name = "ASD"

    # If the tag contains 46 or 52, the device's name is DHPT
    elif re.search(regEx_dhpt, ss_tag):
        device_name = "DHPT"
    
    # If the tag contains DPI, the device's name is DPI
    elif re.search(regEx_dpi, ss_tag):
        device_name = "DPI"
    
    # If the tag contains UB, the device's name is WELL_STATUS
    elif re.search(regEx_ub, ss_tag):
        device_name = "WELL_STATUS"
    
    # If the tag contains UXCF-..A, the device's name is SEMA_COMM
    elif re.search(regEx_uxcfa, ss_tag):
        device_name = "SEMA_COMM"

    # If the tag contains UXCF-..B, the device's name is SEMB_COMM
    elif re.search(regEx_uxcfb, ss_tag):
        device_name = "SEMB_COMM"

    # If the tag contains FZU, the device's name is SUO
    elif re.search(regEx_fzu, ss_tag):
        device_name = "SUO"

    # If the tag contains UN, the device's name is SUO
    elif re.search(regEx_un, ss_tag):
        device_name = "SCM_STATUS"

    # If the tag does not contain 46 or 52 and contain PI or TI, the device's name is "PT" + number
    elif (not re.search(regEx_dhpt, ss_tag)) and (re.search(regEx_pt, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  "PT" + device_num_split[0][1:]

    # If the tag contains "ZI" or "XY", return the valve name
    elif (re.search(regEx_zi_gc_xc, ss_tag)):
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        # Invent the search number for valve
        if re.search(regEx_41_51, ss_tag):
            search_number =  str(device_num_split[0][1:])
        else:
            search_number =  str(device_num_split[0][1:3])
        for item in valves_name_set:
            if search_number  in item:
                device_name = item
                break

    # For the remaining cases
    else:
        split_str = ss_tag.split("-")
        device_num_split = split_str[-1].split("_")
        device_name =  split_str[1] + device_num_split[0][1:]
        # Add the device name to the name set to retrieve for ZI tags
        valves_name_set.add(device_name)
    
    return device_name

In [154]:
regEx_gc = "GC.*(?<!00)$"
regEx_gen_gc = "GC.*00$"
regEx_dash = "-"
def create_tag_name(ss_tag):
    if re.search(regEx_gc, ss_tag):
        tag_name = "GC" + ss_tag[-5:]
    elif re.search(regEx_gen_gc, ss_tag):
        tag_name = "OFF-SCAN ILK.GC" + ss_tag[-5:]
    else:
        regEx_tag_remove = "D\\d{3}-|-1" # Change this to respective well
        tag_name = re.sub(regEx_tag_remove, "", ss_tag)
    return tag_name

In [155]:
# Define columns index in modbus sheet
starting_row = 2
ss_tag_idx = 0
service_idx = 1
modbus_address_idx = 4
data_type_idx = 5
io_type_idx = 6
bit_number_address_idx = 7


# Define writing columns
write_row = 2
write_tag_name_col = 1
write_address_col = 2
write_data_type_col = 3
write_respect_data_type_col = 4
write_client_access_col = 5
write_scan_rate_col = 6

# Regular expression for general interlock
regEx_gen_gc = "GC.*00$"

# Loop through each row in the BDC2_well6 Modbus sheet
for row in ws_bdc2_well6.iter_rows(min_row=starting_row):
    # Write the tag_name column
    if re.search(regEx_gen_gc, row[ss_tag_idx].value):
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = tag_name
    else:
        device_name = create_device_name(row[ss_tag_idx].value)
        tag_name = create_tag_name(row[ss_tag_idx].value)
        device_tag_name = device_name + "." + tag_name
    ws_bdc2_well6_setpoint.cell(row=write_row, column=write_tag_name_col, value=device_tag_name)

    # Write the address column
    # Write the value Address.Bit_Number if the data type is DI or DO. Otherwise, write Address
    address_value = str(row[modbus_address_idx].value) + '.' + str(row[bit_number_address_idx].value) if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else str(row[modbus_address_idx].value)
    address_value = f"\"{address_value}\""
    ws_bdc2_well6_setpoint.cell(row=write_row, column=write_address_col, value=address_value)

    # Write the data type column
    # Write boolean if the data type is DI or DO. Otherwise, if the data type is AI or AO, wirte "Word" for "16 UNSIGNED" and "Float" for "32 FLOAT"
    data_type_value = "Boolean" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "DO") else ("Word" if row[data_type_idx ].value == "16 UNSIGNED" else "Float")
    ws_bdc2_well6_setpoint.cell(row=write_row, column=write_data_type_col, value=data_type_value)

    # Write the respect data type column
    # Always write 1
    respect_data_type_value = 1
    ws_bdc2_well6_setpoint.cell(row=write_row, column=write_respect_data_type_col, value=respect_data_type_value)

    # Write the client access column
    # Write "RO" if the data type is DI or DO. Otherwise, write "R/W"
    client_access_value = "RO" if (row[io_type_idx].value == "DI" or row[io_type_idx].value == "AI") else "R/W"
    ws_bdc2_well6_setpoint.cell(row=write_row, column=write_client_access_col, value=client_access_value)

    # Write the scan rate 
    # Always write 1000
    scan_rate_value = 1000
    ws_bdc2_well6_setpoint.cell(row=write_row, column=write_scan_rate_col, value=scan_rate_value)

    # Write the service
    service_name = row[service_idx].value
    ws_bdc2_well6_setpoint.cell(row=write_row, column=write_description_col, value=service_name)

    # Increase the row by 1
    write_row += 1

# Save the workbook

In [156]:
# Save the workbook
wb.save("Service_testing_BDC1A_2_12_WELLS.xlsx")